# Containerization

## Introduction
Our story began with exploratory data analysis and continued with data cleaning, feature transformations and model training. So far all we have done has taken place on our local machine (or cloud instance, if you will). Suppose we get enough satisfaction and positive feedback from our model. This is where we want to make our model available to outer world and make it easier for our potential customers to access it. We already experimented with model deployment to facilitate this. But what can we do to make the process even more efficient?
Say hello to packaging and shipping. Packaging everything together allows us to be flexible in terms of more convenient deployment and scaling up the model instances quicker when needed. You may have already heard about virtualizations and environments. There are different methodologies and various tools that make the struggle less painful. Some well-known products are [Vagrant](https://www.vagrantup.com), [Kubernetes](https://kubernetes.io) and [Docker](https://www.docker.com). __Docker__ is the tool we'll be utilizing.

<p align="center">
    <img src="./resources/docker.png" />
</p>

Docker is pretty simple and straight forward. All we need to do is to choose a base image and explicitly state which libraries are desired to be installed and which files should be existent (scripts, models, etc). We can even specify any service or script to run on our instance initialization and expose the listening port (that means instant access to our inference model API with zero effort). We do the config once, then create a docker image using the config manifesto; now we can do infinite shippings of that image (known as containers), and run as much instances as we like by pulling and replicating it. Isn't that cool? And you know what's even cooler? All of these steps can be put inside a single file known as ``Dockerfile`` in form of commands and get executed with single docker build call. During this notebook, we'll be examining these commands one by one and create the Dockerfile, build the docker image and initiate a container instance step by step.

<p align="center">
    <img src="./resources/docker-process.png" />
</p>

With all that said, I won't be getting into installing the docker itself. In fact, there are extensive guides & resources out there that explain installing and [getting started](https://www.docker.com/get-started) with it on your local machine. Docker supports Linux, macOS and Windows and installing it has become much easier than what it used to be earlier. Enough talking; considering that you have already setup docker, let's roll up the sleeves and begin 🤩.

## Choosing the Base Image
[Docker hub](https://hub.docker.com) is the official home for millions of ready-to-use docker images. You can find almost anything you like there. Different operating systems and software solutions with multiple versions (tags), all available at your fingertips.

There are couple of points to bear in mind though:
- Some operating systems are better suited & optimized for specific tasks. Try to do your own research before choosing one.
- Always prefer official images to non-official ones, specially if you're serious on pushing your solution to production.
- Based on the OS you choose (type of Linux distro in our case), the built-in package manager could vary and you'll need to adjust shell commands for adding and installing libraries. Even there could be exclusive considerations when it comes to installing a specific package on your chosen distro.

For our use case, we'll need to employ a Linux distribution and I decided to go with [clearlinux](https://hub.docker.com/_/clearlinux), a [high-performant open-source Linux distribution by Intel](https://clearlinux.org). Please note that we can easily find an OS pre-installed with python in docker images hub. Clearlinux is no exception, and the vendor provides [docker images with different python versions](https://hub.docker.com/r/clearlinux/python) running on top of official clearlinux base image.

The docker command for mentioning the base image with our desired python3 version would be:
```Dockerfile
FROM clearlinux/python:3.7
```

## Copying Inference Script & Model Files
Next, we'll copy inference script & model file to our docker image in baking. Let's first make a directory in our root folder and name it as __*'app'*__. This folder will house the mentioned files for inference. Here are the docker commands:
```Dockerfile
# create a new folder named 'app' (if it doesn't exist) in root and set it as the current working directory
WORKDIR /app
```

Ok, here comes the copy stage. If we create our Dockerfile in the same directory where our inference script exists, this is how the COPY command should look like:
```Dockerfile
# copy "inference.py" & "requirements.txt" => the last argument is the destination, which is current working directory
COPY ["inference.py", "requirements.txt", "./"]

# copy model file to working directory with a similar structure => the last argument is the destination in current directory; folder will be created if it doesn't exist
COPY ["model/model.bin", "model/"]
```

## Installing Required Libraries

We already have pip available in our docker image and that gives us the ability to install additional requirement packages. We can either copy the requirements.txt file just like what we did above (found inside __script__ folder of this repo) and run __*pip install -r requirements.txt*__ as a docker RUN command, or simply initiate a docker RUN pip install command having all the required package names:
```Dockerfile
RUN pip install -r requirements.txt
# OR
RUN pip install numpy pandas sklearn xgboost flask waitress
```

That's all the libraries we need to include for an inference run.

## Exposing Port and Specifying Inference App Entry Point
With our files copied & set and required libraries installed, we can move on to the next step. We need to specify the inference file and the port our web server listens to (defined inside reference.py script) for receiving requests. After all, no inference model does wonders without exposure 😉.
```Dockerfile
# expose the port used by web server (defined on inference.py script)
EXPOSE 9696

# run "python inference.py" command as soon as the instance is up
ENTRYPOINT ["python", "inference.py"]
```

## Dockerfile: Putting It All Together
Let's sum up everything we have gathered so far inside a ``Dockerfile``:

```Dockerfile
FROM clearlinux/python:3.7
WORKDIR /app
COPY ["inference.py", "requirements.txt", "./"]
COPY ["model/model.bin", "model/"]
RUN pip install -r requirements.txt
EXPOSE 9696
ENTRYPOINT ["python", "inference.py"]
````

The ``Dockerfile`` with above lines can be found inside __script__ folder of this repository.

## Building the Docker Image
Now that the Dockerfile is ready, we can run the build command and assemble our custom image accordingly. Navigate to __script__ folder of this repo and run the following command on your terminal to build the custom inference docker container:

```shell
docker build -t claims-severity .
```

Notes:
- Pay attention not to miss the dot at the end of the above command.
- You can replace ``claims-severity`` name with anything you like.

The build takes a while to complete, depending on your connection speed.

## Running the Inference Instance
Let's create an instance and run it on our machine:

```shell
docker run -it --rm -p 9000:9696 claims-severity
```

Did you notice the port number difference in our command? I made it different deliberately to make a distinction between local host and container ports. The first port number (9000) belongs to local host machine and the second one (9696) is the container port in which the instance's web server is listening to requests.

That's it! Our inference instance is ready to serve.

## Testing the Docker Instance: Mocking and Sending an Inference Request
If everything goes as described, you should be able to test-run an inference request on the running instance.

In [1]:
# choosing a random sample from test data

import pandas as pd

DATA_PATH = './script/data/'
test_data = pd.read_csv(DATA_PATH+'test_cleaned.csv.gz')

# choose a sample row
# this should have all the columns except the 'id' column which was note used for training
sample = test_data[test_data.columns[~test_data.columns.isin(['id'])]].sample(n=1, random_state=1024).to_json()

In [2]:
# mock an outgoing API request to our prediction endpoint and print the results

import requests

api_url = "http://localhost:9000/predict"

api_response = requests.post(url=api_url, json=sample).json()
print(api_response)

{'prediction': 2284.148193359375}


Congrats, It was a success! Our inference instance works flawlessly :)

## Important Note
As you've noticed so far, we didn't discuss anything about security considerations. The main reason was that I wanted to keep things as simple as possible and not complicate them for starting off. You'll definitely need some more steps to secure the container tightly (and probably limit the API inference calls to only authorized users). But that's another story for another time, and we won't be getting into that for now.